# Example MAE, MSE, RMSE, R-Squared, Bias, and Control Charts

In [1]:
import dash
from dash import Dash, dash_table, html, dcc
import dash_bootstrap_components as dbc
import numpy as np
import statsmodels.api as sm
import sklearn.metrics as metrics
import pandas as pd
import plotly.express as px
from plotly.express import *
import plotly.graph_objects as go
import plotly.figure_factory as ff
from dash.dependencies import Input, Output
import flask
from flask import Flask
import pandas_datareader.data as web
import datetime as dt
import ipywidgets as widgets
from ipywidgets import interact, Dropdown, interact_manual
import warnings
warnings.filterwarnings('ignore')

###### 10Y Rates 

In [2]:
R = 'GS10'
start_date = dt.datetime(2009,1,1)
end_date = dt.datetime.today()

Rates_df = web.DataReader(R, 'fred', start_date, end_date)
Rates_df.reset_index('DATE', inplace=True, drop=False)
Rates_df.dropna(inplace=True)

###### Three different forecasts (could be replaced by Consensus, Weighted Avg, and +1 Std. Dev for example)

### 1. Forecast Method 1

In [3]:
data1 = Rates_df.tail(50)
data1['MA'] = data1['GS10'].rolling(window=5).mean()
data1['MA'] = data1['MA'].shift(1)

f1 = data1[data1['MA'].notna()]
f1['Error'] = round(f1['GS10'].sub(f1['MA'], axis = 0),4)
f1['Absolute Error'] = abs(f1['Error'])

mae_1 = round(metrics.mean_absolute_error(f1['GS10'], f1['MA']),3)
mse_1 = round(metrics.mean_squared_error(f1['GS10'], f1['MA']), 3)
rmse_1 = round(np.sqrt(mse_1), 3)
r2_1 = round(metrics.r2_score(f1['GS10'],f1['MA']), 3)
bias_1 = round(f1['Error'].mean(),3)

### 2. Forecast Method 2

In [4]:
data2 = Rates_df.tail(50)
data2['Smooth'] = round(data2['GS10'].ewm(alpha=0.5).mean(),3)

f2 = data2[data2['Smooth'].notna()]
f2['Error'] = round(f2['GS10'].sub(f2['Smooth'], axis = 0),4)
f2['Absolute Error'] = abs(f2['Error'])

mae_2 = round(metrics.mean_absolute_error(f2['GS10'], f2['Smooth']),3)
mse_2 = round(metrics.mean_squared_error(f2['GS10'], f2['Smooth']), 3)
rmse_2 = round(np.sqrt(mse_2), 3)
r2_2 = round(metrics.r2_score(f2['GS10'],f2['Smooth']), 3)
bias_2 = round(f2['Error'].mean(),3)

### 3. Forecast Method 3

In [5]:
data3 = Rates_df.tail(50)
weights = np.array([0.6, 0.3, 0.1])
sum_weights = np.sum(weights)

data3['WMA'] = (data3['GS10'].rolling(window=3)
                 .apply(lambda x: np.sum(weights*x) / sum_weights, raw=False))
data3['WMA'] = data3['WMA'].shift(1)

f3 = data3[data3['WMA'].notna()]
f3['Error'] = round(f3['GS10'].sub(f3['WMA'], axis = 0),4)
f3['Absolute Error'] = abs(f3['Error'])

mae_3 = round(metrics.mean_absolute_error(f3['GS10'], f3['WMA']),3)
mse_3 = round(metrics.mean_squared_error(f3['GS10'], f3['WMA']), 3)
rmse_3 = round(np.sqrt(mse_3), 3)
r2_3 = round(metrics.r2_score(f3['GS10'],f3['WMA']), 3)
bias_3 = round(f3['Error'].mean(),3)

##### Table
##### It can be passed through a callback so the table would reflect dropdown selection

In [6]:
mae = [mae_1, mae_2, mae_3]
bias = [bias_1, bias_2, bias_3]
mse = [mse_1, mse_2, mse_3]
rmse = [rmse_1, rmse_2, rmse_3]
r2 = [r2_1, r2_2, r2_3]
Forecast = ["Forecast 1", "Forecast 2", "Forecast 3"]

dtable = pd.DataFrame({'Method': Forecast,
                      "MAE": mae,
                      "Bias":bias,
                      "MSE":mse,
                      "RMSE":rmse,
                      "R-Squared":r2})

dtable

,Method,MAE,Bias,MSE,RMSE,R-Squared
0,Forecast 1,0.342,-0.008,0.181,0.425,0.655
1,Forecast 2,0.108,-0.000,0.021,0.145,0.967
2,Forecast 3,0.309,0.001,0.157,0.396,0.732


### Tracking Signal & Control

In [7]:
opts = {
    'Forecast 1': f1,
    'Forecast 2': f2,
    'Forecast 3': f3
}

layout = go.Layout(
    plot_bgcolor='#F0F8FF',
    font_family='Montserrat',
    font_color='#000000',
    font_size=14,
)

def track_sig(df_choice):
    df = opts[df_choice]
    MAE = df['Absolute Error'].expanding().mean()
    df['TS'] = (df['GS10'] - df.iloc[:,2])/MAE
    df['UCL'] = pd.Series([3 for x in range(len(Rates_df.index))])
    df['LCL'] = pd.Series([-3 for x in range(len(Rates_df.index))])
    
    figure = px.line(x=df['DATE'], y=df['TS'], markers=True)
    figure.add_scatter(x=df['DATE'], y=df['UCL'], name = 'UCL')
    figure.add_scatter(x=df['DATE'], y=df['LCL'], name = 'LCL')
    figure.update_layout(title_text ='Tracking Signal',
                         yaxis_title ='Tracking Signal',
                         xaxis_title = 'Time')
    figure.update_layout(layout)
    return figure

interact_manual(
    track_sig,
    df_choice=opts.keys(),
)

interactive(children=(Dropdown(description='df_choice', options=('Forecast 1', 'Forecast 2', 'Forecast 3'), va…

<function __main__.track_sig(df_choice)>

In [8]:
opts = {
    'Forecast 1': f1,
    'Forecast 2': f2,
    'Forecast 3': f3
}

def control(df_choice):
    df = opts[df_choice]
    df['Error_2'] = df['Error']**2
    Error_2 = df['Error_2'].sum()
    N_1 = (df['Error_2'].count())-1
    Sigma = (Error_2/N_1)**(0.5)
    df['UCL_3'] = pd.Series([3*Sigma for x in range(len(Rates_df.index))])
    df['LCL_3'] = pd.Series([-3*Sigma for x in range(len(Rates_df.index))])
    fig = px.line(x=df['DATE'], y=df['Error'], markers=True)
    fig.add_scatter(x=df['DATE'], y=df['UCL_3'], name = 'UCL')
    fig.add_scatter(x=df['DATE'], y=df['LCL_3'], name = 'LCL')
    fig.update_layout(title_text ='Control Chart',
                         yaxis_title ='Errors',
                         xaxis_title = 'Time')
    fig.update_layout(layout)
    return fig

interact_manual(
    control,
    df_choice=opts.keys(),
)

interactive(children=(Dropdown(description='df_choice', options=('Forecast 1', 'Forecast 2', 'Forecast 3'), va…

<function __main__.control(df_choice)>